### Introduction

By now we have seen the GCN and DeepWalk, which when given a graph gave us the embedding for each node in the graph. These embedding can then be used for other purposes like link prediction or classification. Lets move on to a slightly different problem. In this problem we need the embeddings for each node of a graph where new nodes are continously been added. A possible way to do this would be do rerun the entire model (GCN or DeepWalk) again on the new graph, but it is computationaly expensive. Today  we will be covering a new method which will allow us to get embedding for such graphs is a much easier way.

### The start
In the (GCN or DeepWalk) model, the graph was fixed beforehand, lets say the 'Zachary karate club', some model was trained on it, and then we could make prediction regarding which person went to which part of the club after separation.
![Zachary Karate Club](img/karate_club.png "Karate Club")

In this problem the nodes in this graph were fixed from beginning and all the predictions were also to be made on this fixed nodes. In contrast to this, take 'Youtube' videos to be the nodes and assume there is a edge between related videos, and say we have to classify these videos into categories depending on the content. If we take the same model as in the previous dataset, we can classify all these videos, but lets say some new video is added to 'YouTube', if we want to classify it we will have to retrain the model on the entire new dataset again. This is not feasible as there are too many videos for us to retrain.



To solve this issue, what we can do is to not learn embedding for each node but to learn a function which, given the features and edges joining this node, will give the embeddings for the node. 

# Aggregating Neighbours

The idea is to generate embeddings based on the neighbourhood of the given node. In other words, the embedding of a node will depend upon the embedding of the nodes it is connected to. Like in the graph below, the node 1 and 2 are likely to be more similar than node 1 and 5.
![Simple_graph](img/example_graph_1.png "Simple Graph")

Now lets see a simple method to generate embedding depending on neighbours.

First we assign random values to the embeddings and on each step we will make the value of the embedding the average of the embeddings of all the nodes it shares an edge with. The following example shows the working on a simple linear graph.

![Mean_Embeddings](img/animation.gif "Mean Embeddings")

This is a very simple idea, which can be generalized by representing it in the following way
![Simple_Neighbours](img/simple_neighbours.png "Simple Neighbours")

Here The Black Box joining A with B, C , D represents some function of the A,B, C , D. ( In the above animation it was the mean function). We can replace this box by any function like say sum or max. This function is known as the aggregator function.

Now lets try to make it more general by using not only the neighbhours of a node but also the neighbours of neighbours. The first question is how to make use of neighbours of neighbours. The way which we will be using here is to first generate each node's embedding in the first step by using only its neighbours just like we did above, then in the second step we will use these embeddings to generate the new embeddings. Take a look at the following 

![One_Layer_Aggregation](img/aggregation_1.png "Aggregation Demo")

The numbers written along with the nodes are the value of embedding at time, T=0.

Values of embedding after one step are as follows

![Animation_aggregation_layer_1](img/animation_2.gif "Aggregation Layer 1")

So after one interation the values are as follows:

![Second_Layer_Aggregation](img/aggregation_2.png "Aggregation After One Layer")

Repeating the same procedure again on this new graph we get (try verifying yourself)

![Third_Layer_Aggregation](img/aggregation_3.png "Aggregation After Two Layer")

Lets try to do some analysis of the aggregation. Represent by $A^{(0)}$ the initial value of embedding of A(i.e. 0.1), by $A^{(1)}$ the value after one layer(i.e. 0.25) similarly $A^{(2)}$, $B^{(0)}$, $B^{(1)}$ and all other values.

Clearly 

$$A^{(1)} = \frac{(A^{(0)} + B^{(0)} + C^{(0)} + D^{(0)})}{4}$$

Similarly

$$A^{(2)} = \frac{(A^{(1)} + B^{(1)} + C^{(1)} + D^{(1)})}{4}$$

Writing all the value in the RHS in terms of initial values of embeddings we get

$$A^{(2)} = \frac{\frac{(A^{(0)} + B^{(0)} + C^{(0)} + D^{(0)})}{4} + \frac{A^{(0)}+B^{(0)}+C^{(0)}}{3} + \frac{A^{(0)}+B^{(0)}+C^{(0)}+E^{(0)} +F^{(0)}}{5} + \frac{A^{(0)}+D^{(0)}}{2}}{4}$$

If you look closely you will see the all the nodes that were either neighbours of A or neighbours of neighbours of A are present in this term. It is equavalent to saying that all nodes that have distance of less than or equal to 2 edges from A are influencing this term. Had there been a node G connected only to node F. then it is clearly at a distance of 3 from A and hence wont be influencing this term.

Generalizing this we can say that if we repeat this produce N times, then all the nodes ( and only those nodes) that are at a within a distance N from the node will be influencing the value of the terms.

If we replace the mean function, with some other function lets say $F$, then in this case we can write

$$A^{(1)} = F(A^{(0)} , B^{(0)} , C^{(0)} , D^{(0)})$$

Or more generally

$$A^{(k)} = F(A^{(k-1)} , B^{(k-1)} , C^{(k-1)} , D^{(k-1)})$$

If we denote by $N(v)$ the set of neighbours of $v$, so $N(A)=\{B, C, D\}$ and $N(A)^{(k)}=\{B^{(k)}, C^{(k)}, D^{(k)}\}$, the above equation can be simplified as

$$A^{(k)} = F(A^{(k-1)}, N(A)^{(k-1)} )$$

This process can be visualized as:

![Sampling](img/showing_1.png "Showing one")


This method is quite effective in generating the node embeddings. But there is an issue, if a new node is added to the graph how can be get its embeddings? This is an issue that cannot be tackled with this type of model. Clearly something new is needed, but what? 



One alternative that we can try is to replace the function F by multiple functions such that in first layer it is 
F1, in second layer F2 and so on, and then fixing the amount of layers that we want lets say k.

So our embeddings will become like this


![Sampling_2](img/showing_2.png "Showing one")



Lets formalize our notation a bit now, so that it is easy to understand things.

1. Instead of writing $A^{(k)}$  we will be writing $h_{A}^{k}$
2. Rename the functions $F1$, $F2$ and so on as, $AGGREGATE_{1}$, $AGGREGATE_{2}$ and so on. i.e, $Fk$ becomes $AGGREGATE_{k}$.
3. There will be a total of $K$ aggregation functions.
3. Let our graph be represented by $G(V,E)$ where $V$ is the set of vertices and $E$ is the set of edges.

## What GraphSAGE proposes?

What we have been doing by now can be written as 

Initialise($h_{v}^{0}$) $\forall v \in V$ <br>
for $k=1..K$ do <br>
&nbsp;&nbsp;&nbsp;&nbsp;for $v\in V$ do<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$h_{v}^{k}=AGGREGATE_{k}(h_{v}^{k-1}, \{h_{u}^{k-1} \forall u \in N(v)\})$

$h_{v}^{k}$ will now be containing the embeddings

### Some issues with this

Take a look at the sample graph that we discussed above, in this graph even though the initial embeddings for $E$ and $F$ were different but because their neighbours were same they ended with exactly the same embedding, this is not a a good things as their must be atleast some difference between their embeddings. 

GraphSAGE proposes an interesting idea to deal with it. Rather than passing both of the them into the same aggregating function, what we will do is to pass into aggregating function only the neighbours and then concantenating this vector with the vector of that node. This can be written as:

$h_{v}^{k}=CONCAT(h_{v}^{k-1},AGGREGATE_{k}( \{h_{u}^{k-1} \forall u \in N(v)\}))$

In this way we can prevent two vectors from attaining exactly the same embedding.

Lets now add some non linearity to make it more expressive. So it becomes

$h_{v}^{k}=\sigma[W^{(k)}.CONCAT(h_{v}^{k-1},AGGREGATE_{k}( \{h_{u}^{k-1} \forall u \in N(v)\}))]$

Where \sigma is some non linear function (eg. RELU, sigmoid, etc.) and $W^{(k)}$ is the weight matrix, each layer will have one such matrix. If you looked closely, you will have seen that there no trainable parameters till now in our model. the $W$ matrix has been added to have something that model can learn to have some learning in the model.

One more thing we will add is to normalize the value of h after each iteration,i.e, divide them by thier L2 norm, and hence our complete algorithm becomes.

![GraphSAGE_Algorithm](img/graphsage_algorithm.png "GraphSAGE")



To get the model learning, we need the loss function. For the general unsupervised learning problem the problem loss problem serves pretty well.

![Loss](img/Loss_function.png "Loss")


For supervised learning, either we can learn the embeddings first and then use those embeddings for the downstream task or combine both the part of learning embeddings and the part of applying these embeddings in the task into a single end to end models and then use the loss for the final part, and backpropagate to learn the embeddings while solving the task simultaneously.



# Aggregator Architectures
One of the key difference between GCN and Graphsage is the generalisation of the aggregation function which was simply mean aggregation in GCN. So rather than simply taking the average we use generalised aggregation function in GraphSAGE. GraphSAGE owes its inductivity to its aggregator functions.

## Mean aggregator 
Mean aggregator is as simple as you thought it would be. In mean aggregator we simply
take the elementwise mean of the vectors in **{h<sub>u</sub><sup>k-1</sup>     ∀u ∈ N (v)}**.
In other words, we can average embeddings of all nodes in the neighborhood to construct the neighborhood embedding.
![mean aggregator](img/ma.png)

## Pool aggregator
Until now, we were using weighted average type of approach. But we could also use pooling type of approach for example we can do elementwise min or max pooling. So this would be another option where we are taking the messages from our neighbours, transforming them and applying some type of pooling technique(max pooling or min pooling).
![pool aggregator](img/pa.png)

In the above equation max denotes the elementwise max operator and σ is a nonlinear activation function (yes you are right it can be ReLU..). Please note that the function applied before the max pooling can be an arbitrarily deep multi-layer perceptron, but in the original paper simple single-layer architectures is preferred.

## LSTM aggregator
We could also use a deep neural network like LSTM to learn how to aggregate the neighbours. Since LSTM is not order invariant but we want our aggregator to be order invariant so when we train we could try to train this over several random orderings or permutation to make sure that this will learn that order is not important.

# Inductive capability
One of the interesting propertery of GraphSAGE is that we can train our model on one subset of graph and apply this model  on other subset of this graph. The reason we can do this is that we can do parameter sharing i.e. those processing boxes are the same everywhere (W and B are shared across all the computational graphs or architectures). So when a new architecture comes into play we can borrow the parameters (W and B), do a forward pass and we get our prediction. 
![sharing parameters](img/sharing_param.png)

This property of GraphSAGE is very useful in the prediction of protein interaction. For example we can train our model on protein interaction graph from model organism A (left hand side in the figure below) and generate embedding on newly collected data from other model organism say B (right hand side in the figure).

![protein_interaction](img/protein.png)

We know that our old methods like deepwalk were not able to generalise to a new unseen graph. So if any new node gets added to the graph we had to train our model from scratch but since our new method is generalised to the unseen graphs, so in order to predict the embeddings of the new node we just have to make the computational graph of the new node, transfer the parameters to the unseen part of the graph and we can make predictions.

![new node](img/new_node.png)

We can use this property in social-network (like facebook). Consider the first graph in the above figure, users in a social-network are represented by the nodes of the graph. Initially we have users represented by first graph, so we will train our model on this graph. After some time suppose another user is added in the network, now we don't have to train our model from scratch on the second graph, we will just create the computational graph of the new node, borrow the parameters from the already trained model and then we can find the embeddings of the newly added user.

# Implementation in PyTorch

## Imports

In [1]:
import torch
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import time
import random
from sklearn.metrics import f1_score
from collections import defaultdict



## GraphSAGE class

In [2]:

"""
Simple supervised GraphSAGE model as well as examples running the model
on the Cora and Pubmed datasets.
"""

class MeanAggregator(nn.Module):
    """
    Aggregates a node's embeddings using mean of neighbors' embeddings
    """
    def __init__(self, features, cuda=False, gcn=False): 
        """
        Initializes the aggregator for a specific graph.
        features -- function mapping LongTensor of node ids to FloatTensor of feature values.
        cuda -- whether to use GPU
        gcn --- whether to perform concatenation GraphSAGE-style, or add self-loops GCN-style
        """

        super(MeanAggregator, self).__init__()

        self.features = features
        self.cuda = cuda
        self.gcn = gcn
        
    def forward(self, nodes, to_neighs, num_sample=10):
        """
        nodes --- list of nodes in a batch
        to_neighs --- list of sets, each set is the set of neighbors for node in batch
        num_sample --- number of neighbors to sample. No sampling if None.
        """
        # Local pointers to functions (speed hack)
        _set = set
        if not num_sample is None:
            _sample = random.sample
            samp_neighs = [_set(_sample(to_neigh, 
                            num_sample,
                            )) if len(to_neigh) >= num_sample else to_neigh for to_neigh in to_neighs]
        else:
            samp_neighs = to_neighs

        if self.gcn:
            samp_neighs = [samp_neigh + set([nodes[i]]) for i, samp_neigh in enumerate(samp_neighs)]
        unique_nodes_list = list(set.union(*samp_neighs))
        unique_nodes = {n:i for i,n in enumerate(unique_nodes_list)}
        mask = Variable(torch.zeros(len(samp_neighs), len(unique_nodes)))
        column_indices = [unique_nodes[n] for samp_neigh in samp_neighs for n in samp_neigh]   
        row_indices = [i for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]
        mask[row_indices, column_indices] = 1
        if self.cuda:
            mask = mask.cuda()
        num_neigh = mask.sum(1, keepdim=True)
        mask = mask.div(num_neigh)
        if self.cuda:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list).cuda())
        else:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list))
        to_feats = mask.mm(embed_matrix)
        return to_feats

class Encoder(nn.Module):
    """
    Encodes a node's using 'convolutional' GraphSage approach
    """
    def __init__(self, features, feature_dim, 
            embed_dim, adj_lists, aggregator,
            num_sample=10,
            base_model=None, gcn=False, cuda=False, 
            feature_transform=False): 
        super(Encoder, self).__init__()

        self.features = features
        self.feat_dim = feature_dim
        self.adj_lists = adj_lists
        self.aggregator = aggregator
        self.num_sample = num_sample
        if base_model != None:
            self.base_model = base_model

        self.gcn = gcn
        self.embed_dim = embed_dim
        self.cuda = cuda
        self.aggregator.cuda = cuda
        self.weight = nn.Parameter(
                torch.FloatTensor(embed_dim, self.feat_dim if self.gcn else 2 * self.feat_dim))
        init.xavier_uniform(self.weight)

    def forward(self, nodes):
        """
        Generates embeddings for a batch of nodes.
        nodes     -- list of nodes
        """
        neigh_feats = self.aggregator.forward(nodes, [self.adj_lists[int(node)] for node in nodes], 
                self.num_sample)
        if not self.gcn:
            if self.cuda:
                self_feats = self.features(torch.LongTensor(nodes).cuda())
            else:
                self_feats = self.features(torch.LongTensor(nodes))
            combined = torch.cat([self_feats, neigh_feats], dim=1)
        else:
            combined = neigh_feats
        combined = F.relu(self.weight.mm(combined.t()))
        return combined


class SupervisedGraphSage(nn.Module):

    def __init__(self, num_classes, enc):
        super(SupervisedGraphSage, self).__init__()
        self.enc = enc
        self.xent = nn.CrossEntropyLoss()

        self.weight = nn.Parameter(torch.FloatTensor(num_classes, enc.embed_dim))
        init.xavier_uniform(self.weight)

    def forward(self, nodes):
        embeds = self.enc(nodes)
        scores = self.weight.mm(embeds)
        return scores.t()

    def loss(self, nodes, labels):
        scores = self.forward(nodes)
        return self.xent(scores, labels.squeeze())


## Load and Run

In [3]:

def load_cora():
    num_nodes = 2708
    num_feats = 1433
    feat_data = np.zeros((num_nodes, num_feats))
    labels = np.empty((num_nodes,1), dtype=np.int64)
    node_map = {}
    label_map = {}
    with open("./cora/cora.content") as fp:
        for i,line in enumerate(fp):
            info = line.strip().split()
            feat_data[i,:] = [float(x) for x in info[1:-1]]
            node_map[info[0]] = i
            if not info[-1] in label_map:
                label_map[info[-1]] = len(label_map)
            labels[i] = label_map[info[-1]]

    adj_lists = defaultdict(set)
    with open("./cora/cora.cites") as fp:
        for i,line in enumerate(fp):
            info = line.strip().split()
            paper1 = node_map[info[0]]
            paper2 = node_map[info[1]]
            adj_lists[paper1].add(paper2)
            adj_lists[paper2].add(paper1)
    return feat_data, labels, adj_lists

def run_cora():
    np.random.seed(1)
    random.seed(1)
    num_nodes = 2708
    feat_data, labels, adj_lists = load_cora()
    features = nn.Embedding(2708, 1433)
    features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=False)

    agg1 = MeanAggregator(features, cuda=True)
    enc1 = Encoder(features, 1433, 128, adj_lists, agg1, gcn=True, cuda=False)
    agg2 = MeanAggregator(lambda nodes : enc1(nodes).t(), cuda=False)
    enc2 = Encoder(lambda nodes : enc1(nodes).t(), enc1.embed_dim, 128, adj_lists, agg2,
            base_model=enc1, gcn=True, cuda=False)
    enc1.num_samples = 5
    enc2.num_samples = 5

    graphsage = SupervisedGraphSage(7, enc2)
    rand_indices = np.random.permutation(num_nodes)
    test = rand_indices[:1000]
    val = rand_indices[1000:1500]
    train = list(rand_indices[1500:])

    optimizer = torch.optim.SGD(filter(lambda p : p.requires_grad, graphsage.parameters()), lr=0.7)
    times = []
    for batch in range(100):
        batch_nodes = train[:256]
        random.shuffle(train)
        start_time = time.time()
        optimizer.zero_grad()
        loss = graphsage.loss(batch_nodes, 
                Variable(torch.LongTensor(labels[np.array(batch_nodes)])))
        loss.backward()
        optimizer.step()
        end_time = time.time()
        times.append(end_time-start_time)
        print (batch, loss.item())

    val_output = graphsage.forward(val) 
    print ("Validation F1:", f1_score(labels[val], val_output.data.numpy().argmax(axis=1), average="micro"))
    print ("Average batch time:", np.mean(times))


if __name__ == "__main__":
    run_cora()

/home/solsec/anaconda3/envs/GCN/lib/python3.7/site-packages/ipykernel_launcher.py:84: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/home/solsec/anaconda3/envs/GCN/lib/python3.7/site-packages/ipykernel_launcher.py:113: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


0 1.9567843675613403
1 1.9345124959945679
2 1.9103538990020752
3 1.8844720125198364
4 1.8507955074310303
5 1.8135206699371338
6 1.7809829711914062
7 1.7433273792266846
8 1.6806893348693848
9 1.6083475351333618
10 1.507189393043518
11 1.4838045835494995
12 1.4189420938491821
13 1.3774986267089844
14 1.3052990436553955
15 1.2045835256576538
16 1.157833218574524
17 1.0809636116027832
18 0.9366143941879272
19 0.8753049969673157
20 0.8408470153808594
21 0.8278254270553589
22 0.7579611539840698
23 0.7283762693405151
24 0.7688902616500854
25 0.9232689738273621
26 0.7151660919189453
27 0.746150553226471
28 0.7622414827346802
29 0.7632286548614502
30 0.5637921094894409
31 0.6164315938949585
32 0.5251699686050415
33 0.48454004526138306
34 0.46924054622650146
35 0.43252280354499817
36 0.47390344738960266
37 0.5962116122245789
38 0.602088451385498
39 0.6352841258049011
40 0.4754328429698944
41 0.3938028812408447
42 0.33638235926628113
43 0.40148407220840454
44 0.3923985958099365
45 0.3397420942783

## References

- https://cs.stanford.edu/people/jure/pubs/graphsage-nips17.pdf 
- https://www.youtube.com/watch?v=7JELX6DiUxQ
- https://www.youtube.com/watch?v=oQL4E1gK3VU
- https://www.youtube.com/watch?v=MIAbDNAxChI
